## **CUDA STF 教程 - Part 5: 并行构造 (parallel_for 与 launch)**

在前面的部分中，我们学习了如何创建基本任务、管理数据依赖、同步以及控制执行和数据位置。CUDA STF 还提供了更高级的构造原语，使得直接在逻辑数据上编写并行计算内核变得更加简洁和强大。本部分将重点介绍 `parallel_for` 和 `launch` 这两个构造。

本部分主要依据您提供的文档中 "parallel_for construct" (Page 24-29) 和 "launch construct" (Page 30-33) 章节的内容。

### **8. parallel_for 构造 (文档 Page 24-29)**

`parallel_for` 是 CUDASTF 提供的一个辅助构造，用于创建在某个索引空间（通常由逻辑数据的形状定义）上执行操作的 CUDA 核函数（或 CPU 核函数，取决于执行位置）。它简化了常见的数据并行模式的实现。

#### **8.1 parallel_for 的基本结构**

`parallel_for` 构造主要包含四个元素：

1. **执行位置 (Execution Place)**: 指定代码将在哪里执行（例如，`exec_place::device(0)` 或 `exec_place::host()`）。  
2. **形状 (Shape)**: 定义了生成核函数将迭代的索引空间。这通常是某个逻辑数据的形状 (`logical_data_handle.shape()`)，也可以是自定义的 `box` 形状。  
3. **数据依赖集 (Data Dependencies)**: 与普通任务一样，声明所访问的逻辑数据及其访问模式（`read()`, `write()`, `rw()`, `reduce()`）。  
4. **代码体 (Body of Code)**: 使用 `->*` 操作符指定的一个 lambda 函数。这个 lambda 函数就是将在每个索引上执行的核函数体。

**lambda 函数的参数:**

* 对于一个 N 维的形状，lambda 的前 N 个参数是 `size_t` 类型的索引（例如，对于二维形状是 `size_t i, size_t j`）。  
* 后续参数是与 `parallel_for` 中声明的逻辑数据依赖相对应的数据实例（通常是 `slice` 对象）。  
* 如果执行位置是设备，则 lambda 函数需要有 `__device__` (或 `_CCCL_DEVICE_`) 修饰符。

#### **8.2 parallel_for 处理一维数组示例 (文档 Page 24)**

我们将以下代码保存到 `p5_01_parallel_for_1D.cu`。

In [ ]:
%%writefile p5_01_parallel_for_1D.cu
#include <cuda/experimental/stf.cuh>
#include <vector>
#include <iostream>
#include <numeric> // For std::iota and other algorithms if needed

int main() {
    using namespace cuda::experimental::stf;
    context ctx;

    const size_t N_array = 128;
    std::vector<int> host_A_vec(N_array);

    auto lA = ctx.logical_data(host_A_vec.data(), host_A_vec.size());
    lA.set_symbol("A_1D_pfor");

    // 在当前设备上执行 parallel_for，迭代 lA 的形状 (0 到 N_array-1)
    // lA 以只写模式访问
    ctx.parallel_for(exec_place::current_device(),
                     lA.shape(),
                     lA.write())
        ->*[] __host__ __device__ (size_t i, slice<int> sA_kernel_arg) -> void {
            // 这是核函数体，会在每个索引 i 上执行
            sA_kernel_arg(i) = 2 * (int)i + 1;
        };

    ctx.finalize();

    // 验证结果 (数据会从设备写回到 host_A_vec)
    bool correct = true;
    for(size_t i = 0; i < N_array; ++i) {
        if (host_A_vec[i] != (2 * (int)i + 1)) {
            correct = false;
            std::cout << "Mismatch at index " << i << ": host_A_vec[" << i << "] = " << host_A_vec[i]
                      << " (Expected: " << (2 * (int)i + 1) << ")" << std::endl;
            break;
        }
    }
    if (correct) {
        std::cout << "parallel_for 1D example: Correct!" << std::endl;
    } else {
        std::cout << "parallel_for 1D example: Incorrect!" << std::endl;
    }

    return 0;
}


Writing p5_01_parallel_for_1D.cu


编译并运行 `p5_01_parallel_for_1D.cu`:
*(注意: 您可能需要根据您的 GPU 修改 `-arch=sm_86` 参数。)*

In [7]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p5_01_parallel_for_1D.cu -o p5_01_parallel_for_1D -arch=sm_86 -lcuda
!./p5_01_parallel_for_1D


parallel_for 1D example: Correct!


**请打开您本地的 [`stf/01-axpy-parallel_for.cu`](./stf/01-axpy-parallel_for.cu)。** 这个示例应该展示了如何使用 `parallel_for` 来实现 AXPY 操作。将其与我们之前讨论的基于 `cuda_kernel` 的 AXPY 实现进行比较。您可以尝试编译运行它：

In [ ]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/01-axpy-parallel_for.cu -o p5_stf_axpy_pfor -arch=sm_86 -lcuda
!./p5_stf_axpy_pfor

#### **8.3 parallel_for 处理多维数组示例 (文档 Page 25)**

对于多维数据形状，`parallel_for` 的 lambda 函数会接收对应数量的索引参数。我们将以下代码保存为 `p5_02_parallel_for_2D.cu`。

In [ ]:
%%writefile p5_02_parallel_for_2D.cu
#include <cuda/experimental/stf.cuh>
#include <vector>
#include <iostream>
#include <tuple> // For std::tuple

int main() {
    using namespace cuda::experimental::stf;
    context ctx;

    const size_t M = 4; // 例如 4 行
    const size_t K = 3; // 例如 3 列
    std::vector<double> host_X_matrix_vec(M * K);

    // 创建一个二维 slice 来描述这个矩阵
    // 使用简化的 make_slice 语法创建连续的 2D slice
    auto sX_host = make_slice(host_X_matrix_vec.data(), M, K);
    auto lX_matrix = ctx.logical_data(sX_host);
    lX_matrix.set_symbol("X_2D_pfor");

    ctx.parallel_for(exec_place::current_device(),
                     lX_matrix.shape(), // 迭代 lX_matrix 的二维形状
                     lX_matrix.write())
        ->*[] _CCCL_DEVICE (size_t i, size_t j, auto sX_kernel_arg) {
            // i 是行索引 (0 到 M-1), j 是列索引 (0 到 K-1)
            sX_kernel_arg(i, j) = (double)(i * 10 + j);
        };

    ctx.finalize();

    // 首先打印实际的数组内容来调试
    std::cout << "Array contents after kernel execution:" << std::endl;
    for(size_t idx = 0; idx < M * K; ++idx) {
        std::cout << "host_X_matrix_vec[" << idx << "] = " << host_X_matrix_vec[idx] << std::endl;
    }

    // 验证 - slice 使用列主序存储，所以 sX_kernel_arg(i,j) 对应 host_X_matrix_vec[j*M + i]
    bool correct = true;
    for(size_t i = 0; i < M; ++i) {
        for(size_t j = 0; j < K; ++j) {
            size_t col_major_index = j * M + i;  // 列主序索引
            double expected_value = (double)(i * 10 + j);
            if (std::abs(host_X_matrix_vec[col_major_index] - expected_value) > 1e-9) {
                 correct = false;
                 std::cout << "Mismatch at (" << i << "," << j << "): host_X_matrix_vec[" << col_major_index << "] = "
                           << host_X_matrix_vec[col_major_index] << " (Expected: " << expected_value << ")" << std::endl;
                 break;
            }
        }
        if (!correct) break;
    }
    if (correct) {
         std::cout << "parallel_for 2D example: Correct!" << std::endl;
    } else {
        std::cout << "parallel_for 2D example: Incorrect!" << std::endl;
    }

    return 0;
}


Writing p5_02_parallel_for_2D.cu


编译并运行 `p5_02_parallel_for_2D.cu`:

In [11]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p5_02_parallel_for_2D.cu -o p5_02_parallel_for_2D -arch=sm_86 -lcuda
!./p5_02_parallel_for_2D

../cccl/cudax/include/cuda/experimental/__stf/stream/interfaces/slice.cuh(245) [device 0] CUDA error in data_copy: invalid pitch argument (cudaErrorInvalidPitchValue).


**请打开您本地的 [`stf/parallel_for_2D.cu`](./stf/parallel_for_2D.cu)。** 这个示例会更完整地展示二维 `parallel_for` 的应用。尝试编译并运行它：

In [12]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/parallel_for_2D.cu -o p5_stf_parallel_for_2D -arch=sm_86 -lcuda
!./p5_stf_parallel_for_2D

#### **8.4 box 形状 (文档 Page 26-27)**

有时，迭代的索引空间不直接对应于某个逻辑数据的形状。对于这些情况，CUDASTF 提供了 `box<size_t dimensions = 1>` 模板类，允许用户定义具有显式边界的多维形状。

* **基于范围的 box**: `box<2>({dim0_extent, dim1_extent})` 表示一个二维迭代空间，第一个索引从 0 到 `dim0_extent-1`，第二个索引从 0 到 `dim1_extent-1`。
  ```cpp
  // ctx.parallel_for(exec_place::current_device(), box<2>({2, 3})) // 迭代 i=0..1, j=0..2
  //     ->*[&](size_t i, size_t j) __device__ {
  //         printf("Box extent: %ld, %ld\n", i, j);
  // };
  ```
* **基于上下界的 box**: `box<2>({{lower0, upper0}, {lower1, upper1}})`。下界包含，上界不包含。
  ```cpp
  // ctx.parallel_for(exec_place::current_device(), box<2>({{5, 8}, {2, 4}})) // i=5..7, j=2..3
  //     ->*[&](size_t i, size_t j) __device__ {
  //         printf("Box bounds: %ld, %ld\n", i, j);
  // };
  ```

#### **8.5 reduce() 访问模式 (文档 Page 27-29)**

`parallel_for` 支持 `reduce()` 访问模式，这使得在 `parallel_for` 生成的计算核函数内部实现归约操作成为可能。

`reduce()` 接受的参数：
1. **归约操作符 (Reduction Operator)**: 定义了如何组合多个值以及如何初始化一个值（例如，求和归约会将两个值相加，并将初始值设为0）。这些操作符定义在 `cuda::experimental::stf::reducer` 命名空间中。  
2. **可选的 `no_init{}` 标签**: 如果提供此标签，则归约结果将累加到逻辑数据中已存在的值上（类似于 `rw()` 访问模式）。默认情况下（不提供 `no_init{}`），逻辑数据的内容将被归约结果覆盖（类似于 `write()` 访问模式）。对没有有效实例的逻辑数据（例如，仅从形状定义的）使用 `no_init{}` 会导致错误。  
3. **其他参数**: 与其他访问模式相同，例如数据位置。

只能对数据接口定义了 `owning_container_of` trait 类的逻辑数据应用 `reduce()` 访问模式。`scalar_view<T>` 数据接口就是这种情况，它将 `owning_container_of` 设置为 `T`。传递给 `parallel_for` lambda 的归约参数是对该类型对象的引用。

**点积示例 (来自文档 Page 28):**
我们将以下代码保存到 `p5_03_dot_reduce_pfor.cu`。

In [1]:
%%writefile p5_03_dot_reduce_pfor.cu
#include <cuda/experimental/stf.cuh>
#include <vector>
#include <numeric> // For std::iota
#include <iostream>
#include <cmath> // For std::abs

int main() {
    using namespace cuda::experimental::stf;
    context ctx;

    const size_t N_dot = 1024;
    std::vector<double> h_x_dot(N_dot), h_y_dot(N_dot);
    std::iota(h_x_dot.begin(), h_x_dot.end(), 1.0);
    std::iota(h_y_dot.begin(), h_y_dot.end(), 1.0);

    auto lX_dot = ctx.logical_data(h_x_dot);
    auto lY_dot = ctx.logical_data(h_y_dot);
    auto lsum_dot = ctx.logical_data(shape_of<scalar_view<double>>());

    lX_dot.set_symbol("X_for_dot_reduce");
    lY_dot.set_symbol("Y_for_dot_reduce");
    lsum_dot.set_symbol("Sum_result_dot_reduce");

    ctx.parallel_for(exec_place::current_device(),
                     lY_dot.shape(), // 迭代空间
                     lX_dot.read(),
                     lY_dot.read(),
                     lsum_dot.reduce(reducer::sum<double>{}) // 对 lsum_dot 进行求和归约
                    )
        ->*[&](size_t i, slice<const double> sX, slice<const double> sY, double& sum_ref) __device__ {
            // sum_ref 是对 lsum_dot 设备实例中用于累加的部分的引用
            // CUDASTF 会处理线程间的归约细节
            sum_ref += sX(i) * sY(i);
        };

    double dot_product_result = ctx.wait(lsum_dot);
    ctx.finalize();

    std::cout << "Dot product (via parallel_for.reduce and ctx.wait): " << dot_product_result << std::endl;

    double expected_dot_product = 0;
    for(size_t i = 0; i < N_dot; ++i) expected_dot_product += h_x_dot[i] * h_y_dot[i];
    std::cout << "Expected dot product: " << expected_dot_product << std::endl;

    if (std::abs(dot_product_result - expected_dot_product) < 1e-9 * expected_dot_product) {
        std::cout << "Dot product example: Correct!" << std::endl;
    } else {
        std::cout << "Dot product example: Incorrect!" << std::endl;
    }

    return 0;
}

Writing p5_03_dot_reduce_pfor.cu


编译并运行 `p5_03_dot_reduce_pfor.cu`:

In [2]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p5_03_dot_reduce_pfor.cu -o p5_03_dot_reduce_pfor -arch=sm_86 -lcuda
!./p5_03_dot_reduce_pfor

Dot product (via parallel_for.reduce and ctx.wait): 3.58438e+08
Expected dot product: 3.58438e+08
Dot product example: Correct!


预定义的归约操作符 (文档 Page 28-29):  
`sum`, `product`, `maxval`, `minval`, `logical_and`, `logical_or`, `bitwise_and`, `bitwise_or`, `bitwise_xor`。  
用户也可以定义自己的归约操作符 (文档 Page 29)。  
**请打开您本地的 [`stf/09-dot-reduce.cu`](./stf/09-dot-reduce.cu) 和 [`stf/word_count_reduce.cu`](./stf/word_count_reduce.cu) (如果存在)。** 这些示例会展示 `reduce()` 的实际应用。

编译并运行 [`stf/09-dot-reduce.cu`](./stf/09-dot-reduce.cu):

In [1]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/09-dot-reduce.cu -o p5_stf_dot_reduce -arch=sm_86 -lcuda
!./p5_stf_dot_reduce

编译并运行 [`stf/word_count_reduce.cu`](./stf/word_count_reduce.cu) (如果文件存在):

In [4]:
# 您需要确保 stf/word_count_reduce.cu 文件存在于您的目录中
# 如果存在，取消下面的注释来编译和运行
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/word_count_reduce.cu -o p5_stf_word_count_reduce -arch=sm_86 -lcuda
!./p5_stf_word_count_reduce

Got 65 words.


### **9. launch 构造 (文档 Page 30-33)**

`ctx.launch` 原语是 CUDASTF 中一种核函数启动机制，它隐式地处理单个核函数到执行位置的映射和启动。与 `parallel_for`（在每个索引点应用相同操作）不同，`launch` 执行的是一个基于**线程层级 (thread hierarchy)** 的结构化计算核函数。

#### **9.1 launch 的基本语法**

```cpp
// ctx.launch([thread_hierarchy_spec], // 可选的线程层级规范
//            [execution_place],       // 执行位置
//            logicalData1.accessMode(),
//            logicalData2.accessMode(), ...)
//     ->*[capture_list] __device__ (thread_hierarchy_spec_t th, // 线程层级对象
//                                   auto data1, auto data2...) {
//     // 核函数实现
// };
```
`launch` 构造包含五个主要元素：  
1.  **可选的执行策略/线程层级规范 (Execution Policy / Thread Hierarchy Specification)**: 显式指定启动形状。例如，指定一组独立线程或可同步线程。  
2.  **执行位置 (Execution Place)**: 指示代码将在哪里执行。  
3.  **数据依赖集 (Data Dependencies)**: 与其他任务构造类似。  
4.  **代码体 (Body of Code)**: 使用 `->*` 指定的 lambda 函数，带有 `__device__` 修饰符。  
5.  **线程层级参数 (`thread_info` 或 `thread_hierarchy_spec_t th`)**: lambda 的第一个参数，用于查询线程属性（如全局ID、线程总数）和层级结构。

**示例 (来自文档 Page 30，稍作调整):**
我们将以下代码保存到 `p5_04_launch_axpy.cu`。

In [11]:
%%writefile p5_04_launch_axpy.cu
#include <cuda/experimental/stf.cuh>
#include <vector>
#include <iostream>
#include <cmath> // For std::sin, std::cos, std::abs
#include <numeric> // For std::iota

// LAUNCH_N 和 LAUNCH_ALPHA 可以在 main 中定义并通过捕获列表传递给 lambda 
// 或者作为逻辑数据传递。这里我们为了简化，作为全局常量。
// 但在实际应用中，推荐通过捕获或逻辑数据传递。
// const size_t LAUNCH_N_GLOBAL = 1024; // Example, better to pass via capture
// const double LAUNCH_ALPHA_GLOBAL = 2.5; // Example, better to pass via capture

int main() {
    using namespace cuda::experimental::stf;
    context ctx;

    const size_t LAUNCH_N = 1024;
    const double LAUNCH_ALPHA = 2.5;

    std::vector<double> h_x_launch(LAUNCH_N), h_y_launch(LAUNCH_N), h_y_expected(LAUNCH_N);
    for(size_t i=0; i<LAUNCH_N; ++i) {
        h_x_launch[i] = std::sin((double)i);
        h_y_launch[i] = std::cos((double)i);
        h_y_expected[i] = std::cos((double)i) + LAUNCH_ALPHA * std::sin((double)i); // Precompute expected Y
    }

    auto lX_launch = ctx.logical_data(h_x_launch.data(), LAUNCH_N);
    auto lY_launch = ctx.logical_data(h_y_launch.data(), LAUNCH_N);
    lX_launch.set_symbol("X_for_launch");
    lY_launch.set_symbol("Y_for_launch");

    ctx.launch(par(LAUNCH_N), // 线程层级：LAUNCH_N 个并行线程
               exec_place::current_device(),
               lX_launch.read(), 
               lY_launch.rw()
              )
        ->*[=] __device__ (auto t, slice<const double> x_slice, slice<double> y_slice) {
            size_t tid = t.rank(); // 获取当前线程的全局唯一ID
            // size_t nthreads = t.get_num_threads(); // 获取此 launch 启动的总线程数
            // In this simple case with par(LAUNCH_N) and vector size LAUNCH_N, tid directly maps to index.
            // For more general cases (e.g., if par_size < vector_size), a grid-stride loop is needed:
            // for (size_t ind = tid; ind < x_slice.size(); ind += nthreads) {
            //    y_slice(ind) += LAUNCH_ALPHA * x_slice(ind);
            // }
            if (tid < x_slice.size()) { // Check bounds, x_slice.size() should be LAUNCH_N here
                 y_slice(tid) += LAUNCH_ALPHA * x_slice(tid);
            }
        };

    ctx.finalize();

    // 验证
    bool correct = true;
    for(size_t i=0; i < LAUNCH_N; ++i) {
        if (std::abs(h_y_launch[i] - h_y_expected[i]) > 1e-9) {
            correct = false;
            std::cout << "Mismatch at index " << i << ": Y_launch[" << i << "] = " << h_y_launch[i] 
                      << ", Expected: " << h_y_expected[i] << std::endl;
            break;
        }
    }
    if (correct) {
        std::cout << "launch AXPY example: Correct!" << std::endl;
    } else {
        std::cout << "launch AXPY example: Incorrect!" << std::endl;
    }
    std::cout << "launch example completed." << std::endl;

    return 0;
}

Overwriting p5_04_launch_axpy.cu


编译并运行 `p5_04_launch_axpy.cu`:

In [2]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include p5_04_launch_axpy.cu -o p5_04_launch_axpy -arch=sm_86 -lcuda
!./p5_04_launch_axpy

launch AXPY example: Correct!
launch example completed.


#### **9.2 描述线程层级 (文档 Page 31)**

线程层级规范描述了核函数的并行结构。层级大小可以自动计算、动态指定或编译时指定。

* **`par(num_threads)`**: 并行组 (parallel group)，线程独立执行，不可组内同步。  
  * `par<128>()`: 静态指定大小。  
* **`con(num_threads)`**: 并发组 (concurrent group)，组内线程可以使用 `sync()` API 进行同步（组级别屏障）。  
* **层级嵌套**: 可以嵌套多个组，例如 `par(128, con<256>())` 表示128个独立的组，每个组包含256个可同步的线程。  
* **共享内存**: `con(256, mem(64))` 表示256个线程的组，共享64字节的内存（由STF自动分配在适当的内存层级）。  
* **硬件范围亲和性 (Hardware Scope Affinity)**: 可以指定线程组映射到特定的机器层级。  
  * `hw_scope::thread`: CUDA 线程。  
  * `hw_scope::block`: CUDA 块。  
  * `hw_scope::device`: CUDA 设备。  
  * `hw_scope::all`: 整台机器。  
  * 示例: `par(hw_scope::device | hw_scope::block, par<128>(hw_scope::thread))`

#### **9.3 操作线程层级对象 (th) (文档 Page 32-33)**

传递给 `launch` 核函数体的线程层级对象（通常命名为 `th` 或 `t`）提供了查询层级结构和线程交互的方法：

* `th.rank()`: 线程在整个层级中的全局排名。  
* `th.size()`: 整个层级的总线程数。  
* `th.rank(level_idx)`: 线程在第 `level_idx` 层的排名。  
* `th.size(level_idx)`: 第 `level_idx` 层的线程数。  
* `th.is_synchronizable(level_idx)`: 检查第 `level_idx` 层是否可同步。  
* `th.sync(level_idx)`: 同步第 `level_idx` 层的所有线程。  
* `th.sync()`: 同步最顶层（0级）的所有线程。  
* `th.get_scope(level_idx)`: 获取第 `level_idx` 层的硬件范围亲和性。  
* `th.template storage<T>(level_idx)`: 获取与第 `level_idx` 层关联的本地存储（作为 `slice<T>`）。  
* `th.depth()`: (constexpr) 层级的深度。  
* `th.inner()`: 获取移除了最顶层后的线程层级子集。

**请打开您本地的 [`stf/launch_sum.cu`](./stf/launch_sum.cu)、[`stf/launch_scan.cu`](./stf/launch_scan.cu) 或 [`stf/launch_histogram.cu`](./stf/launch_histogram.cu)。** 这些示例会展示 `launch` 构造以及线程层级操作的实际应用，通常用于实现比简单数据并行更复杂的并行模式（例如，自定义的归约、扫描等）。

In [6]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include -I../cccl/cub -I../cccl/thrust launch_scan.cu -o launch_scan -arch=sm_86 -lcuda
!./launch_scan

1.00 GB in 273.807373 ms (3.73986 GB/s)


### **动手试试:**

1. **编译并运行上面提供的 `parallel_for` (1D `p5_01_parallel_for_1D.cu` 和 2D `p5_02_parallel_for_2D.cu`) 以及 `launch` 的示例代码 (`p5_04_launch_axpy.cu`)。** 确保您理解它们的行为和输出。
2. **研究 [`stf/01-axpy-parallel_for.cu`](./stf/01-axpy-parallel_for.cu)**: 将其与 `stf/01-axpy-cuda_kernel.cu` (来自Part 1)进行比较。`parallel_for` 如何简化了 AXPY 的实现？(您已在前面编译运行过 `p5_stf_axpy_pfor`)
3. **研究 [`stf/09-dot-reduce.cu`](./stf/09-dot-reduce.cu) (您已编译为 `p5_stf_dot_reduce`):**
   * 它是如何使用 `parallel_for` 和 `lsum.reduce(reducer::sum<double>{})` 来计算点积的？  
   * `sum_ref` 在 lambda 中是如何工作的？  
   * `ctx.wait(lsum_dot)` 如何用于获取最终结果？  
4. **(挑战)** 尝试修改 [`stf/09-dot-reduce.cu`](./stf/09-dot-reduce.cu)，使用不同的归约操作符，例如 `reducer::maxval<double>{}` 来找两个向量对应元素乘积的最大值。然后重新编译运行 `p5_stf_dot_reduce`。
5. **研究 [`stf/launch_sum.cu`](./stf/launch_sum.cu)**:  
   * `launch` 的线程层级是如何定义的？  
   * 核函数体是如何使用线程层级对象 `th` 来执行求和的？是否有使用 `th.sync()` 或共享内存？
   尝试编译并运行它:

In [2]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include -I../cccl/cub -I../cccl/thrust ./stf/launch_sum.cu -o p5_stf_launch_sum -arch=sm_86 -lcuda
!./p5_stf_launch_sum

   同样地，您可以尝试编译和运行 [`stf/launch_scan.cu`](./stf/launch_scan.cu) 和 [`stf/launch_histogram.cu`](./stf/launch_histogram.cu) (如果它们存在于您的 `stf` 目录中)。

In [ ]:
# !nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/launch_scan.cu -o p5_stf_launch_scan -arch=sm_86 -lcuda
# !./p5_stf_launch_scan

# !nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/launch_histogram.cu -o p5_stf_launch_histogram -arch=sm_86 -lcuda
# !./p5_stf_launch_histogram

我们已经学习了 CUDA STF 中强大的 `parallel_for` 和 `launch` 构造，它们使得表达复杂并行模式更为便捷。

在教程的 Part 6，我们将讨论 `cuda_kernel` 和 `cuda_kernel_chain` (虽然之前已提及，但会再次回顾其在整体结构中的位置)，以及 STF 如何通过 C++ 类型系统来增强代码的健壮性，最后是模块化使用 STF 的一些高级技巧。